In [1]:

from pydantic_ai.providers.google_gla import GoogleGLAProvider
from pydantic_ai.models.gemini import GeminiModel
from dotenv import load_dotenv
import os
import nest_asyncio
nest_asyncio.apply()

load_dotenv()

google_api_key=os.getenv('google_api_key')

llm=GeminiModel('gemini-2.0-flash', provider=GoogleGLAProvider(api_key=google_api_key))


In [2]:
api_keys={'openai_api_key':os.getenv('openai_api_key')}

In [1]:
from pydantic_ai.mcp import MCPServerStreamableHTTP, MCPServerSSE, MCPServerStdio
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
from pydantic_ai import Agent
from pydantic_ai.messages import ModelMessage
from dataclasses import dataclass

@dataclass
class Message_state:
    messages:list[ModelMessage]

class agent_MCP:
    def __init__(self, mcp_server_url:str, api_keys:dict):
        self.mcp_server_url=mcp_server_url
        self.api_keys=api_keys

        self.memory=Message_state(messages=[])
        self.llm=OpenAIModel('gpt-4.1-mini',provider=OpenAIProvider(api_key=api_keys['openai_api_key']))
        self.mcp_server_url=MCPServerStreamableHTTP(self.mcp_server_url)
        self.mcp_server_stdio=MCPServerStdio('npx', [ '-y','@modelcontextprotocol/server-memory'])
        self.agent=Agent(self.llm, mcp_servers=[self.mcp_server_url, self.mcp_server_stdio])
        self._mcp_context_manager = None
        self._is_connected = False

    async def connect(self):
        """Establish persistent connection to MCP server"""
        if not self._is_connected:
            self._mcp_context_manager = self.agent.run_mcp_servers()
            await self._mcp_context_manager.__aenter__()
            self._is_connected = True
            print("Connected to MCP server")

    async def disconnect(self):
        """Close the MCP server connection"""
        if self._is_connected and self._mcp_context_manager:
            await self._mcp_context_manager.__aexit__(None, None, None)
            self._is_connected = False
            self._mcp_context_manager = None
            print("Disconnected from MCP server")
    
    async def chat(self, query: str):
        """Execute query using persistent MCP connection"""
        if not self._is_connected:
            await self.connect()
        
        result = await self.agent.run(query, message_history=self.memory.messages)
        self.memory.messages = result.all_messages()
        return result.output

    def reset(self):
        """Reset conversation memory"""
        self.memory.messages = []

    async def __aenter__(self):
        """Async context manager entry"""
        await self.connect()
        return self

    async def __aexit__(self, exc_type, exc_val, exc_tb):
        """Async context manager exit"""
        await self.disconnect()

In [7]:
agent2=agent_MCP(mcp_server_url=os.getenv('mcp_server'), api_keys=api_keys)


In [8]:
await agent2.connect()  # Connect once
result1 = await agent2.chat("Hello, what can you help me with?")
print("Result 1:", result1)

NotImplementedError: 